# ARC | CompressARC + Easiest-First Strategy (FIXED)

**Fixed version: submission.json 생성 보장**

References:
- [暗黑AGI](https://www.kaggle.com/code/boristown/agi-compressarc)
- [ARC-AGI Without Pretraining](https://www.kaggle.com/code/iliao2345/arc-agi-without-pretraining)

In this notebook:
* The simpler tasks, which are possible to solve, are prioritized to solve first and have the longer training time than difficult tasks
* The number of training epochs is based on the simplicity score, and is different for each task
* The simplicity score is determined by the amount of colors and pixels in the input

Expected Score: **4.58** (Gold Medal 🥇)

In [ ]:
import random
import numpy as np
import torch
import os

GLOBAL_SEED = 42
fake_mode = not os.getenv('KAGGLE_IS_COMPETITION_RERUN')

def set_all_seeds(seed=GLOBAL_SEED):
    """设置所有可能的随机种子来确保可重现性"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # 为了完全确定性，禁用CUDA的非确定性算法
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # 设置Python哈希种子
    os.environ['PYTHONHASHSEED'] = str(seed)

#set_all_seeds()

### Imports

In [ ]:
import os
import sys
import time
import json
import importlib
import multiprocessing
from multiprocessing import Pool

import numpy as np
import torch

sys.path.append('/kaggle/input/publiccompressarc')

# This little block of code does "import preprocessing" but avoids a name collision with another module
module_path = "/kaggle/input/publiccompressarc/preprocessing.py"
module_name = "preprocessing"
spec = importlib.util.spec_from_file_location(module_name, module_path)
preprocessing = importlib.util.module_from_spec(spec)
sys.modules[module_name] = preprocessing
spec.loader.exec_module(preprocessing)
import train
import arc_compressor
import initializers
import multitensor_systems
import layers
import solution_selection
import visualization
import solve_task

### Getting all the task names, setting defaults and constants

In [ ]:
multiprocessing.set_start_method('spawn', force=True)
torch.set_default_dtype(torch.float32)
torch.set_default_device('cuda')
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

if __name__ == '__main__':

    start_time = time.time()
    end_time = start_time + 12*3600 - 600

    n_cpus = multiprocessing.cpu_count()
    n_gpus = torch.cuda.device_count()

    # Find all the puzzle names
    split = "evaluation" if fake_mode else "test"
    with open(f'../input/arc-prize-2025/arc-agi_{split}_challenges.json', 'r') as f:
        problems = json.load(f)
    task_names = list(problems.keys())
    n_tasks = len(task_names)

    # Compute and sort the easy tasks to train
    simplicty_scores = list()
    for i in range(n_tasks):
        input_matrix = np.array(list(problems.values())[i]['test'][0]['input'])
        unique_values, counts = np.unique(input_matrix, return_counts=True)
        
        color_score = 1 - len(unique_values) / 11
        pixel_score = 1 - (input_matrix.shape[0]*input_matrix.shape[1] / (31*31))
        simplicty_score = color_score*10+pixel_score + 1
        simplicty_scores.append(simplicty_score)
    simplicty_scores = np.sqrt(np.array(simplicty_scores))
    sorted_taskid = np.argsort(-simplicty_scores)
    del problems

### Function that can spawn processes and schedule them on GPUs to take up each GPUs quota

In [ ]:
def parallelize_runs(gpu_quotas, task_usages, n_iteration_list, verbose=False):
    gpu_quotas = gpu_quotas[:]
    # Schedule the tasks greedily to max out memory usage
    t = time.time()
    tasks_started = [False for i in range(n_tasks)]
    tasks_finished = [False for i in range(n_tasks)]
    processes = [None for i in range(n_tasks)]
    process_gpu_ids = [None for i in range(n_tasks)]
    with multiprocessing.Manager() as manager:
        memory_dict = manager.dict()
        solutions_dict = manager.dict()
        error_queue = manager.Queue()
        while not all(tasks_finished):
            if not error_queue.empty():
                raise ValueError(error_queue.get())
            for i in sorted_taskid: #range(n_tasks): #
                if tasks_started[i] and not tasks_finished[i]:
                    processes[i].join(timeout=0)
                    if not processes[i].is_alive():
                        tasks_finished[i] = True
                        gpu_quotas[process_gpu_ids[i]] += task_usages[i]
                        if verbose:
                            print(task_names[i], 'finished on gpu', process_gpu_ids[i],
                                  'New quota is', gpu_quotas[process_gpu_ids[i]])
            for gpu_id in range(n_gpus):
                for i in sorted_taskid: #range(n_tasks): #
                    enough_quota = gpu_quotas[gpu_id] > task_usages[i]
                    enough_cpus = sum(map(int, tasks_started)) - sum(map(int, tasks_finished)) < n_cpus
                    if not tasks_started[i] and enough_quota and enough_cpus:
                        gpu_quotas[gpu_id] -= task_usages[i]
                        args = (task_names[i], split, end_time, n_iteration_list[i], gpu_id, memory_dict, solutions_dict, error_queue)
                        #args = (task_names[i], split, end_time, n_iteration_list, gpu_id, memory_dict, solutions_dict, error_queue)
                        p = multiprocessing.Process(target=solve_task.solve_task, args=args)
                        p.start()
                        processes[i] = p
                        tasks_started[i] = True
                        process_gpu_ids[i] = gpu_id
                        if verbose:
                            print(task_names[i], 'started on gpu', process_gpu_ids[i],
                                  'New quota is', gpu_quotas[process_gpu_ids[i]])
            time.sleep(1)
        if not error_queue.empty():
            raise ValueError(error_queue.get())
        memory_dict = dict(memory_dict)
        solutions_dict = dict(solutions_dict)
    time_taken = time.time() - t
    if verbose:
        print('All jobs finished in', time_taken, 'seconds.')
    return memory_dict, solutions_dict, time_taken

### Measuring the amount of memory used for every task

In [ ]:
if __name__ == '__main__':
    gpu_memory_quotas = [torch.cuda.mem_get_info(i)[0] for i in range(n_gpus)]

    gpu_task_quotas = [int(gpu_memory_quota // (4 * 1024**3)) for gpu_memory_quota in gpu_memory_quotas]
    task_usages = [1 for i in range(n_tasks)]
    memory_dict, _, _ = parallelize_runs(gpu_task_quotas, task_usages, 2*np.ones(n_tasks, dtype=int), verbose=False)
    #memory_dict, _, _ = parallelize_runs(gpu_task_quotas, task_usages, 2, verbose=False)
    
    # Sort the tasks by decreasing memory usage
    tasks = sorted(memory_dict.items(), key=lambda x: x[1], reverse=True)
    task_names, task_memory_usages = zip(*tasks)

### Computing the time taken, while saturating memory

In [ ]:
if __name__ == '__main__':
    #test_steps = 2000
    test_steps = 500 if fake_mode else 2000
    iterations_list = (1.0+simplicty_scores*test_steps/sum(simplicty_scores)).astype(int) #2000 is the total number of iterations for distributing
    
    safe_gpu_memory_quotas = [memory_quota - 6 * 1024**3 for memory_quota in gpu_memory_quotas]

    #_, _, time_taken = parallelize_runs(safe_gpu_memory_quotas, task_memory_usages, test_steps, verbose=False)
    _, _, time_taken = parallelize_runs(safe_gpu_memory_quotas, task_memory_usages, iterations_list, verbose=False)

### Computing the solution for every task, while saturating memory and time

In [ ]:
if __name__ == '__main__':
    time_per_step = time_taken / test_steps
    time_left = end_time - time.time()
    # n_steps = 5 if fake_mode else int(time_left // time_per_step)
    # _, solutions_dict, time_taken = parallelize_runs(safe_gpu_memory_quotas, task_memory_usages, n_steps, verbose=True)
    
    #test_steps = int(2.0 * time_left // time_per_step)
    test_steps = 500 if fake_mode else int(2.0 * time_left // time_per_step) #change iteration number here
    iterations_list = (1.0+simplicty_scores*test_steps/sum(simplicty_scores)).astype(int) #2000 is the total number of iterations for distributing\
    _, solutions_dict, time_taken = parallelize_runs(safe_gpu_memory_quotas, task_memory_usages, iterations_list, verbose=True)
    
    # Format the solutions and put into submission file
    with open('submission.json', 'w') as f:
        json.dump(solutions_dict, f, indent=4)
        
    print(n_tasks, 'tasks solved.')
    #print(n_steps, 'steps taken.')
    print(time_taken, 'seconds taken.')
    
    # IMPORTANT: Ensure submission.json is preserved
    print(f"\n✅ submission.json created successfully!")
    print(f"✅ File size: {os.path.getsize('submission.json')} bytes")
    print(f"✅ Task count: {len(solutions_dict)}")

### IMPORTANT: Visualization 코드 제거됨

**submission.json 생성을 보장하기 위해 visualization 코드를 제거했습니다.**

이제:
- Run All → submission.json 생성 ✓
- Submit to Competition → 정상 제출 ✓